# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

## Formulation of the optimal control problem: Pendubot

### Model description
<img src="https://raw.github.com/wiki/mayataka/CGMRES/images/fig_pendubot.png" width=25%>
The above figure shows the model of a pendubot. The equation of the motion can be derived by the Lagrange's method as
$$ \ddot{\theta}_1 = \ddot{\theta}_1 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) ,$$
$$ \ddot{\theta}_2 = \ddot{\theta}_2 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) .$$
We ommit to describe its details here because it is too complicated.
We assume that the input of the pendubot is limited as  
$$u_{\rm min} \leq u \leq u_{\rm max} , $$
where $u_{\rm min}, u_{\rm max} \in \mathbb{R}$.

### Formulate the optimal control problem
We define the state vector as
$$ x = \begin{bmatrix} \theta_1 \\ \theta_2 \\ \dot{\theta}_1 \\ \dot{\theta}_2 \end{bmatrix} .$$
The state equation is then given as 
$$\dot{x} = f(t, x, u) = \begin{bmatrix} \dot{\theta}_1 \\ \dot{\theta}_2 \\ \ddot{\theta}_1 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) \\ \ddot{\theta}_2 (\theta_1, \theta_2, \dot{\theta}_1, \dot{\theta}_2) \end{bmatrix} .$$
To apply NMPC based on the C/GMRES method, we transform the inequality constraints into equality constraints. This is done by introducing a virtual control input $u_{\rm dummy} \in \mathbb{R}$ to this optimal control problem and imosing the equality constraint
$$C(t, x, u) = u^2 + u_{\rm dummy}^2 - (\frac{u_{\rm max} - u_{\rm min}}{2})^2 = 0, $$
where $u_{\rm dummy} \in \mathbb{R}$.  
Next, we set the cost function 
$$ J = \varphi(t, x) + \int_{t}^{t+T} L(t, x, u) .$$
The objective is to invert the pendubot and stabilize it. In other words, we want to converge the state of the pendubot to
$$x_{\rm ref} := \begin{bmatrix} \pi \\ 0 \\ 0 \\ 0 \end{bmatrix}.$$  
To achieve this goal, we set the terminal cost as
$$\varphi(t,x) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q_{\rm terminal} (x - x_{\rm ref}) ,$$  
where $Q_{\rm terminal} = {\rm diag}\left\{ {q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4 \right\}$ and ${q_{\rm terminal}}_1, {q_{\rm terminal}}_2, {q_{\rm terminal}}_3, {q_{\rm terminal}}_4$ are positive real constants. We also set the stage cost as  
$$L(t, x, u) = \frac{1}{2} (x - x_{\rm ref})^{\rm T} Q (x - x_{\rm ref}) + \frac{1}{2} r u^2 - r_{\rm dummy} u_{\rm dummy},$$  
where $Q = {\rm diag} \left\{ q_1, q_2, q_3, q_4 \right\}$ with positive real constants $q_1, q_2, q_3, q_4$, $r$ is positive real constant, and $r_{\rm dummy}$ is so, too. Note that the term $- r_{\rm dummy} u_{\rm dummy}$ is to determine the sign of $u_{\rm dummy}$ uniquely.


### Numerical solver of NMPC
In the following example, we compute the solution of NMPC using the multiple shooting based C/GMRES method with condensing of variables with respect to the constraints on the saturation of the control input. That is, $C(t, x, u) = u^2 + u_{\rm dummy}^2 - (\frac{u_{\rm max} - u_{\rm min}}{2})^2 = 0, $ is condensed.

---

# Generate C++ codes of NMPC model  

## Import modules

In [ ]:
from sympy import *
from autogenu import symbolic_functions as symfunc
from autogenu import cpp_generator as gencpp

## Set dimensions
`dimx`: Dimension of the state vector $x$   
`dimu`: Dimension of the control input vector $u$  
`dimc`: Dimension of the constraints $C(x, u) = 0$  

In [ ]:
dimx = 4
dimu = 1
dimc = 0

## Generate t, x, u, and lmd, necessary variables for the formulation of the optimal control problem
`t`: Time parameter $t$  
`x`: The state vector $x$  
`u`: The control input vector $u$  
`lmd`: The Lagrange multiplier for the state equation $\lambda$

In [ ]:
t = Symbol('t')  
x = symbols('x[0:%d]' %(dimx))  
u = symbols('u[0:%d]' %(dimu+dimc))
lmd = symbols('lmd[0:%d]' %(dimx))

## Define variables used in the state equation, constraints, and cost function
- Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

- Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

- Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
m1, m2, l1, l2, d1, d2, J1, J2, g = symbols('m1, m2, l1, l2, d1, d2, J1, J2, g')

# You can also define functions used  in the state function or in the cost funciton 

## Define weight parameters used in the stage cost and the terminal cost
`q`: The weight array for the state in the stage cost  
`r`: The weight array for the control input in the stage cost  
`q_terminal`: The weight array for the state in the terminal cost  
`x_ref`: The reference value of the state

In [ ]:
q = symbols('q[0:%d]' %(dimx))
r = symbols('r[0:%d]' %(dimu+dimc))
q_terminal = symbols('q_terminal[0:%d]' %(dimx))
x_ref = symbols('x_ref[0:%d]' %(dimx))

## Define the state equation, constraints, the stage cost, and the terminal cost
`fxu`: The state equation $ f(t, x, u)$  
`Cxu`: The equality constraisnts $C(t, x, u) $  
`L`: The stage cost $L(t, x, u)$  
`phi`: The terminal cost $\phi (t, x)$  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = [x[2], 
        x[3],
       -(sin(x[0] + x[1]) * d2 * g * m2 + g * (d1 * m1 + l1 * m2) * sin(x[0]) - 0.2e1 * d2 * (x[2] + x[3] / 0.2e1) * l1 * x[3] * m2 * sin(x[1]) - u[0]) / (0.2e1 * d2 * m2 * l1 * cos(x[1]) + d1 * d1 * m1 + d2 * d2 * m2 + l1 * l1 * m2 + J1 + J2), 
       (g * d2 * l1 * m2 * (d1 * m1 + l1 * m2) * sin(x[0] - x[1]) / 0.2e1 - d2 * d2 * g * l1 * m2 * m2 * sin(x[0] + 0.2e1 * x[1]) / 0.2e1 - (d1 * d1 * m1 - d1 * l1 * m1 / 0.2e1 + l1 * l1 * m2 / 0.2e1 + J1) * m2 * g * d2 * sin(x[0] + x[1]) - l1 * l1 * m2 * m2 * d2 * d2 * (pow(x[2], 0.2e1) + x[2] * x[3] + pow(x[3], 0.2e1) / 0.2e1) * sin(0.2e1 * x[1]) - l1 * m2 * d2 * ((pow(x[2] + x[3], 0.2e1) * d2 * d2 + pow(x[2], 0.2e1) * l1 * l1) * m2 + (d1 * d1 * m1 + J1 + J2) * pow(x[2], 0.2e1) + 0.2e1 * J2 * x[2] * x[3] + J2 * pow(x[3], 0.2e1)) * sin(x[1]) + g * (d2 * d2 * l1 * m2 * m2 / 0.2e1 + (d1 * d2 * d2 * m1 + J2 * l1) * m2 + J2 * d1 * m1) * sin(x[0]) - u[0] * (d2 * m2 * l1 * cos(x[1]) + d2 * d2 * m2 + J2)) / (0.2e1 * d2 * m2 * l1 * cos(x[1]) + d1 * d1 * m1 + d2 * d2 * m2 + l1 * l1 * m2 + J1 + J2) / (d2 * d2 * m2 + J2)]

# Define the constraints (if dimc > 0)
Cxu = []

# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 + sum(r[i] * u[i]**2 for i in range(dimu))/2 

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2

## Generate the optimality conditions
`Hamiltonian`: $H(t, x, u, \lambda) := L(t, x, u) + \lambda^{\rm T} f(t, x, u)$  
`hx`: The partial derivartive of the Hamiltonian $H (t, x, u, \lambda)$ with respect to $x$, i.e., $\left(\frac{\partial H}{\partial x} \right)^{\rm T} (t, x, u, \lambda)$    
`hu`: The partial derivartive of the Hamiltonian $H (t, x, u, \lambda)$ with respect to $u$, i.e., $\left(\frac{\partial H}{\partial u} \right)^{\rm T}, (t, x, u, \lambda)$    
`phix`: The partial derivative of the terminal cost $\phi(x)$ with respect to $x$, i.e., $\left(\frac{\partial \phi}{\partial x} \right)^{\rm T} (t, x)$

In [ ]:
if(dimc > 0):
    Hamiltonian = L + symfunc.dot_product(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc))
else:
    Hamiltonian = L + symfunc.dot_product(lmd, fxu) 

phix = symfunc.diff_scalar_func(phi, x)
hx = symfunc.diff_scalar_func(Hamiltonian, x)
hu = symfunc.diff_scalar_func(Hamiltonian, u)

## Symplify phix, hx, and hu
Note: if `hx` and `hu` is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
Set all parameters used in the state equation, constraints, and the cost function

In [ ]:
scalar_parameters = [[m1, 0.2], [m2, 0.7], 
                     [l1, 0.3], [l2, 0.3], 
                     [d1, 0.15], [d2, 0.257], 
                     [J1, 0.006], [J2, 0.051], 
                     [g, 9.80665]]

array_parameters = [['q', dimx, '{1, 1, 0.1, 0.1}'], 
                    ['r', dimu, '{0.1}'], 
                    ['q_terminal', dimx, '{1, 1, 0.1, 0.1}'], 
                    ['x_ref', dimx, '{M_PI, 0, 0, 0}']]

## Generate C++ codes of NMPC model
generate `nmpc_model.hpp` and `nmpc_model.cpp` in a directory of `model_name`

In [ ]:
model_name = "pendubot"

gencpp.make_model_dir(model_name)
gencpp.generate_cpp(dimx, dimu, dimc, fxu, Cxu, phix, hx, hu, model_name)
gencpp.generate_hpp(dimx, dimu, dimc, scalar_parameters, array_parameters, model_name)

---  
# Generate C++ codes for numerical simulation  

## Import modules

In [ ]:
from autogenu import solver_parameters as slvprm
from autogenu import initialization_parameters as iniprm
from autogenu import simulation_parameters as simprm
from autogenu import cpp_executor as cppexe
from autogenu import simulation_plottor as simplt
from autogenu import animation_generator as animgen
init_printing()

## Set solvers  
set which solvers you use  in `solver_index`
  
1.  `ContinuationGMRES` : The continuation/GMRES method (the original C/GMRES method, single shooting).
2.  `MultipleShootingCGMRES` : The multiple shooting based C/GMRES method with condensing of the state and the Lagragne multipliers with respect to the state equation.
3.  `MSCGMRESWithSaturation` : The multiple shooting based C/GMRES method with condensing of the state, the Lagragne multipliers with respect to the state equation, and variables with respect to the constraints on the saturation function on the control input.

In [ ]:
solver_index = 3

## Set saturaions on the control input if you choose `solver_index = 3`
- Saturation on the control input: $u_{i, {\rm min}} \leq u_i \leq u_{i, {\rm max}}$  
$u_i \in \mathbb{R}$ : a constrained component of the control input $u$  
- Transformed equality constraint: $(u_i - \frac{u_{i, {\rm max}} + u_{i, {\rm min}}}{2})^2 - ( \frac{u_{i, {\rm max}} - u_{i, {\rm min}}}{2})^2 + {u_d}_i ^2 = 0$  
${u_d}_i \in \mathbb{R}$ : a dummy input for the transformation of the saturation into the equality constraint  
- Additional term in the stage cost $L(x, u)$ with respect to the saturation of $u_i$: $- {r_d}_i {u_d}_i + \frac{1}{2} {r_q}_i {u_d}_i ^2$  
   ${r_d}_i > 0$: a weight parameter to avoid failure of numerical computation, ${r_q}_i \geq 0$: a weight parameter to increase mergin of the saturation  

`index`: $i$  
`u_min`: $u_{i, {\rm min}}$  
`u_max`: $u_{i, {\rm max}}$   
`dummy_weight`: ${r_d}_i > 0$  
`quadratic_weight` :  ${r_q}_i \geq 0$  
`saturation_param` = [`index`, `u_min`, `u_max`, `dummy_weight`, `quadratic_weight`]

In [ ]:
saturation_list = [[0, -5, 5, 1.0e-01, 0]]
# Define saturation_list as follows.
# saturation_list = [[index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    .., 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight]

## Set parameters for the solver

`T_f`, `alpha`: Parameters for the length of the horizon $T(t)$: $T(t) = T_f (1 - e^{-\alpha t})$.  
`N`: The division number of the horzion for the numerical computation.  
`zeta`: A stabilization parameter of the C/GMRES method. It may work well to set `zeta` by the reciprocal of the sampling time.
`finite_difference_step`: Step length of a finite difference approximations of hessian-vector products in C/GMRES.   
`kmax`: The maximam number of the iteration of the GMRES.

In [ ]:
T_f = 1.0
alpha = 1.0
N = 50
zeta = 1000
finite_difference_step = 1.0e-08
kmax = 3

solver_parameters = slvprm.SolverParameters(T_f, alpha, N, zeta, finite_difference_step, kmax)

## Set parameters for the initialization of the solution  
`initial_guess_solution`: The initial guess of the solution of the optimal control problem (OCP) for initialization of the solution of NMPC.   
`residual_torelance`: The residual torelance of the solution of the OCP for the initialization of the solution of NMPC. The Newton iteration terminates when the optimality error is less than this velue.  
`max_iteration`: The maxmum number of Newton iteration for the initialization of the solution of NMPC.  
`Lagrange_multiplier`: An optional parameter for `MSCGMRESWithSaturation`. This is a part of the initial guess of the solution, the initial guess of the Lagrange multiplier with respect the constraints on the saturation function of the control input.

In [ ]:
initial_guess_solution = [0.0]
residual_torelance = 1.0e-06
max_iteration = 50
Lagrange_multiplier = [0.01]

if solver_index == 1 or solver_index == 2:
    initialization_parameters = iniprm.InitializationParameters(initial_guess_solution, 
                                                                residual_torelance, 
                                                                max_iteration)
else:
    initialization_parameters = iniprm.InitializationParameters(initial_guess_solution, 
                                                                residual_torelance, 
                                                                max_iteration, 
                                                                Lagrange_multiplier)

## Set parameters for numerical simulation
`initial_time`: Initial time of the numerical simulation.  
`initial_state`: Initial state vector of the system.  
`simulation_time`: Simulation time of the numerical simulation.  
`sampling_time`: The sampling time of the numerical simulation.

In [ ]:
initial_time = 0  
initial_state = [0, 0, 0, 0]  
simulation_time = 10
sampling_time = 0.001  

simulation_parameters = simprm.SimulationParameters(initial_time, 
                                                    initial_state, 
                                                    simulation_time, 
                                                    sampling_time)

## Generate main.cpp and CMakeLists.txt

In [ ]:
if solver_index == 1 or solver_index == 2:
    gencpp.generate_main(solver_index, 
                         solver_parameters, 
                         initialization_parameters, 
                         simulation_parameters, 
                         model_name)
else:
    gencpp.generate_main(solver_index, 
                         solver_parameters, 
                         initialization_parameters, 
                         simulation_parameters, 
                         model_name, 
                         saturation_list)    

gencpp.generate_cmake(solver_index, model_name)
gencpp.generate_cmake_for_model(model_name)

## Build and run simulation
NOTE: If you use Windows OS and an error occurs in `cppexec.setCMake(model_name)`, you may solve that error by running codes which are commented out instead of the original codes.

In [ ]:
cppexe.set_cmake(model_name)
cppexe.make_and_run(model_name)
# cppexe.remove_build_dir(model_name)
# cppexe.set_cmake(model_name, MSYS=True)
# cppexe.make_and_run(model_name)

## Plot the simulation results

In [ ]:
plot = simplt.SimulationPlottor(model_name)
plot.set_scales(2,5,2)
# plot.show_plots()
plot.save_plots()

## Draw an animation of the simulation results

In [ ]:
anim = animgen.TwoLinkArm(model_name)
anim.set_skip_frames(10)
anim.generate_animation()